In [1]:
import os, re, itertools
import pandas as pd
import nltk

In [2]:
path_to_gs = "C:\\Program Files\\gs\\gs10.03.1\\bin"
os.environ['PATH'] += os.pathsep + path_to_gs

In [3]:
df = pd.read_csv("D:\\New folder\\ML\\Tweets.csv")
df = df[['text', 'airline_sentiment']]

In [4]:
r = re.compile(r"([@])(\w+)\b")
AllReferences = map(lambda x: r.findall(x), df['text'])

AllUniqueReferencesCombined = set(list(itertools.chain.from_iterable(AllReferences)))
References = map(lambda x:x[0] + x[1], AllUniqueReferencesCombined)

file = open("References.txt", 'a')
for each in References:
    file.write(each+"\n")
file.close()

In [5]:
def GetNounPhrases(s):
    try:
        sentences = nltk.sent_tokenize(s)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        sentences = [nltk.pos_tag(sent) for sent in sentences]
    except:
        return[]
    else:
        grammar = r"NP:{<DT><NN|NNS|NNP|NNPS>*<NN|NNS|NNP|NNPS>}"
        cp = nltk.RegexpParser(grammar)
        noun_phrases_list = [[''.join(leaf[0] for leaf in tree.leaves())
                             for tree in cp.parse(sent).subtrees()
                             if tree.label() == 'NP']
                             for sent in sentences]
        return noun_phrases_list

In [9]:
for group, sub in df.groupby('airline_sentiment'):
    noun_phrases = map(lambda x:GetNounPhrases(x), sub['text'])
    noun_phrases = list(itertools.chain.from_iterable(noun_phrases))
    AllNounPhrases = set(list(itertools.chain.from_iterable(noun_phrases)))
    filename = "Noun Phrases for" + str(group) + "Review.txt"
    with open(filename, 'a', encoding='utf-8') as file:
        for each in AllNounPhrases:
            file.write(each + "\n")
    file.close()